In [1]:
import torch
#import torchvision

import pandas as pd


In [2]:

cifar10n_labels = torch.load("CIFAR-10_human.pt")

print(cifar10n_labels.keys())
print(cifar10n_labels["worse_label"][:10])

classname_dict = {
    "airplane": 0,
    "automobile": 1,
    "bird": 2,
    "cat": 3,
    "deer": 4,
    "dog": 5,
    "frog": 6,
    "horse": 7,
    "ship": 8,
    "truck": 9
}
# reverse the dictionary
classname_dict = {v: k for k, v in classname_dict.items()}


dict_keys(['clean_label', 'aggre_label', 'worse_label', 'random_label1', 'random_label2', 'random_label3'])
[4 9 0 5 1 9 2 4 8 8]


In [3]:
for split in ["train"]:
    dataset_list = []
    count_dict = {}

    # combine train and test labels
    #labels = trainset.targets

    for i,_ in enumerate(cifar10n_labels["clean_label"]):
        label = cifar10n_labels["clean_label"][i]
        classname = classname_dict[label]
        count_dict[classname] = count_dict.get(classname, 0) + 1

        # if count_dict[classname] == 1:
        #     # save the image
        #     img = torchvision.transforms.ToTensor()(trainset.data[i])
        #     torchvision.utils.save_image(img, f"{classname}-{count_dict[classname]:04}.png")
        
        if split in ["val","test"]: source_folder = "test"
        else: source_folder = "train"
        filename = f"cifar10/{source_folder}/{classname}/{count_dict[classname]:>04}.png"
        dataset_list.append(
            {
                "filename": filename,
                #"label_orig": label,
                "label_worse": cifar10n_labels["worse_label"][i],
                "label_clean": cifar10n_labels["clean_label"][i],
                "label_agg": cifar10n_labels["aggre_label"][i],
                "label_random1": cifar10n_labels["random_label1"][i],
                "label_random2": cifar10n_labels["random_label2"][i],
                "label_random3": cifar10n_labels["random_label3"][i],
            }
        )

    print(dataset_list[:10])

    # create dataframe
    df = pd.DataFrame(dataset_list)

    print(f"Found {len(df)} images in total")

    df.set_index("filename", inplace=True)

    # load txt file data/benchmark_imglist/cifar10/train_cifar10.txt
    train_df = pd.read_csv(f"../../benchmark_imglist/cifar10/{split}_cifar10.txt", sep=" ", header=None)

    # rename columns to match
    train_df.columns = ["filename", "label_orig"]

    # update index of both dataframes
    train_df.set_index("filename", inplace=True)
    
    print(f"Found {len(train_df)} images in {split} set")

    #sort both dataframes by index
    #train_df.sort_index(inplace=True)
    #df.sort_index(inplace=True)
    print(train_df.head())
    print(df.head())

    # create new dataframe with updated labels but the order of the train_df
    #train_df_noisy = pd.concat([train_df, df], axis=1, join="inner")
    train_df_noisy = train_df.merge(df, how="left", left_index=True, right_index=True)
    print(train_df_noisy.head())

    # remove column 1
    train_df_noisy.drop(columns=["label_clean"], inplace=True)

    #print(train_df.head())
    #print(train_df_noisy.head())
    # remove rows from df_new that are not in df
    train_df_noisy = train_df_noisy[train_df_noisy.index.isin(train_df.index)]

    print(f"After merging, the number of rows in the train_df_noisy is {len(train_df_noisy)}")

    # save the new dataframes as txt files, selecting the label_worse column
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar10/{split}_cifar10n_worse.txt", sep=" ", header=False, columns=["label_worse"])
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar10/{split}_cifar10n_agg.txt", sep=" ", header=False, columns=["label_agg"])
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar10/{split}_cifar10n_random1.txt", sep=" ", header=False, columns=["label_random1"])
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar10/{split}_cifar10n_random2.txt", sep=" ", header=False, columns=["label_random2"])
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar10/{split}_cifar10n_random3.txt", sep=" ", header=False, columns=["label_random3"])

[{'filename': 'cifar10/train/frog/0001.png', 'label_worse': 4, 'label_clean': 6, 'label_agg': 6, 'label_random1': 6, 'label_random2': 4, 'label_random3': 6}, {'filename': 'cifar10/train/truck/0001.png', 'label_worse': 9, 'label_clean': 9, 'label_agg': 9, 'label_random1': 9, 'label_random2': 9, 'label_random3': 9}, {'filename': 'cifar10/train/truck/0002.png', 'label_worse': 0, 'label_clean': 9, 'label_agg': 9, 'label_random1': 9, 'label_random2': 9, 'label_random3': 0}, {'filename': 'cifar10/train/deer/0001.png', 'label_worse': 5, 'label_clean': 4, 'label_agg': 4, 'label_random1': 4, 'label_random2': 5, 'label_random3': 4}, {'filename': 'cifar10/train/automobile/0001.png', 'label_worse': 1, 'label_clean': 1, 'label_agg': 1, 'label_random1': 1, 'label_random2': 1, 'label_random3': 1}, {'filename': 'cifar10/train/automobile/0002.png', 'label_worse': 9, 'label_clean': 1, 'label_agg': 1, 'label_random1': 1, 'label_random2': 1, 'label_random3': 9}, {'filename': 'cifar10/train/bird/0001.png',